# Imports

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

As before, we can change rcParams, to list all of them just call the plt.rcParams.keys() method of the dictionary.

In [ ]:
print(plt.rcParams.keys())

In [ ]:
try:
    from cycler import cycler
except:
    pass
# example usage:
# cycler('color', ['r', 'g', 'b', 'y']) + cycler('lw', [1, 2, 3])

plt.rcParams['patch.linewidth'] = 0.5
#plt.rcParams['patch.facecolor'] = '348ABD'  # blue
#plt.rcParams['patch.edgecolor'] = 'EEEEEE'
plt.rcParams['patch.antialiased'] = True

plt.rcParams['font.size'] = 12.0

plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['axes.edgecolor'] = 'black'
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['axes.grid'] = False
plt.rcParams['axes.titlesize'] = 'large'
plt.rcParams['axes.labelsize'] = 'large'
plt.rcParams['axes.labelcolor'] =  '555555'
#axes.axisbelow: True       # grid/ticks are below elements (eg lines, text)


# want to define these colors:
# E24A33 : red, 348ABD : blue, 988ED5 : purple, 777777 : gray
# FBC15E : yellow, 8EBA42 : green, FFB5B8 : pink

colorlist = ['#E24A33', '#348ABD', '#988ED5', '#777777', '#FBC15E', '#8EBA42', '#FFB5B8']
#colornames = ['red', 'blue', 'purple', 'gray', 'yellow', 'green', 'pink']
try:
    # newer Matplotlib 

    plt.rcParams['axes.prop_cycle'] = cycler('color', colorlist)
except:
    # older Matplotlib (<1.5 I think)
    plt.rcParams['axes.color_cycle'] = colorlist

#xtick.color: 555555
#xtick.direction: in

#ytick.color: 555555
#ytick.direction: in

plt.rcParams['grid.color'] = 'grey'
plt.rcParams['grid.linestyle'] =  '-'

plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['figure.edgecolor'] = 'white'

plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.default'] = 'regular'

Same as before

In [ ]:
import astropy
import astropy.coordinates as coordinates
import astropy.units as u

import astropy.constants as c
from astropy.table import Table

# Plotting images

## Popular packages

    -WCSAxes
    http://wcsaxes.readthedocs.org/
      
    - APLpy
    https://aplpy.github.io/
    http://aplpy.readthedocs.org/
    
    -PyWCSGrid2 (discontinued!)
    http://leejjoon.github.io/pywcsgrid2/

In [ ]:
from astropy.wcs import WCS
from astropy.io import fits

from wcsaxes import datasets
import aplpy
import pywcsgrid2 as pwg

## Read in the data

In [ ]:
hdu_msx = fits.open('data/msx.fits')[0]
wcs_msx = WCS(hdu_msx.header)

hdu_l1558_13co = fits.open('data/l1558_13co.fits')[0]
wcs_l1558_13co = WCS(hdu_l1558_13co.header)

hdu_bolocam = fits.open('data/bolocam.fits')[0]
wcs_bolocam = WCS(hdu_bolocam.header)

hdu_rosat = fits.open('data/rosat.fits')[0]
wcs_rosat = WCS(hdu_rosat.header)

hdu_2mass = fits.open('data/2mass_k.fits')[0]
wcs_2mass = WCS(hdu_2mass.header)


## aplpy

Aplpy has a rather different approach than the other packages. It has its own commands for most things. Each plot is a layer.

In [ ]:
gc = aplpy.FITSFigure('data/2mass_k.fits')
gc.show_rgb('data/2MASS_arcsinh_color.png')

gc.show_contour('data/mips_24micron.fits', colors='white')

data = np.loadtxt('data/yso_wcs_only.txt')
ra, dec = data[:, 0], data[:, 1]

gc.show_markers(ra, dec, layer='marker_set_1', edgecolor='r',
                facecolor='none', marker='o', s=10, alpha=0.5)


We can also slice data, for image cubes this is very convenient. The $^{13}$CO data for L1558 is a data cube.

In [ ]:
f = aplpy.FITSFigure('data/l1558_13co.fits', slices=[20],
                     figsize=(5,5))
f.show_colorscale()
f.add_grid()

Since we can also change the slicing, we can quickly create a PV-diagram.

In [ ]:
f = aplpy.FITSFigure('data/l1558_13co.fits', dimensions=[2, 1],
                     slices=[50], figsize=(5,5))
f.show_colorscale(aspect='auto')
f.add_grid()
ax.tick_labels.set_xformat('%.1f');

## Exercise

    - Add contours to the  L1558 maps.
    - Change the coordinate display system.
    - Change the color map the L1558 maps.

## WCSAxes

WCSAxes is intended as a replacement for both APLpy, PyWCSGrid2, trying to take the best from both and is developed by many of the same people. It ties into Matplotlib just like pywcsgrid2.

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(4,4))
ax = fig.add_axes([0.1, 0.1, 0.9, 0.9], projection=wcs_msx)

Now we have an axes object just like we usually do with Matplotlib. This is much more convenient, since you do not have to learn special methods and objects, just to change things in the plot.

We can first check the histogram of the whole image, to see what range in values we have.

In [ ]:
_ = plt.hist(hdu_msx.data.flatten(),bins=50);
plt.yscale('log')

Seems most values are around $0$, and $5\times10^{-4}$.

In [ ]:
fig = plt.figure(figsize=(15,10))

ax1 = fig.add_subplot(131, projection=wcs_msx)
ax1.imshow(hdu_msx.data, vmin=-1e-5, vmax=2.e-4, cmap=plt.cm.inferno,
          origin='lower')

ax2 = fig.add_subplot(132, projection=wcs_msx)
ax2.imshow(hdu_msx.data, vmin=-1e-5, vmax=2.e-4, cmap=plt.cm.viridis,
          origin='lower')

ax3 = fig.add_subplot(133, projection=wcs_msx)
ax3.imshow(hdu_msx.data, vmin=-1e-5, vmax=2.e-4, cmap=plt.cm.jet,
          origin='lower')
fig.subplots_adjust(wspace=0.3)


### Exercise

    - Mark some coordinates of interest in the map.
    - Try different color maps.
    - Try different vmin and vmax.
    - Change coordinates to Equatorial.\
    - Add axis labels

### A more complicated figure

In [ ]:
fig = plt.figure(figsize=(15,10))

ax = fig.add_subplot(111, projection=wcs_msx)
ax.imshow(hdu_msx.data, vmin=-1e-5, vmax=2.e-4, cmap=plt.cm.viridis,
          origin='lower')

# change display of coordinates
xcoord = ax.coords[0]
ycoord = ax.coords[1]
xcoord.set_ticks(number=4)
xcoord.set_major_formatter('hh:mm:ss.s')
xcoord.set_separator(('$^{\sf h}$', "'", '"'))

# add contours from a different data set
ax.contour(hdu_bolocam.data, transform=ax.get_transform(WCS(hdu_bolocam.header)),
           levels=[1,2,3,4,5,6], colors='white');

# mark some points of interest
tr_gal = ax.get_transform("galactic")
l = [-0.01, 0.015, 0.10, 0.16]
b = [0.02, 0.035, 0.02, -0.06]
ax.scatter(l, b, transform=tr_gal, s=40,
           edgecolor='white', facecolor='r', alpha=0.5)

# overlay a second coordinate system.
overlay = ax.get_coords_overlay('fk5')

ax.set_xlim(-0.5, 148.5)
ax.set_ylim(-0.5, 148.5);

### Exercise

    - Change images, and overlay different layers of the L1558 $^{13}$CO cube as contours with different colors.
    - Change contour colors.
    - If you have your own data, load that in and display it.
    - Add axis labels.

## ROSAT

All sky plots again...

In [ ]:
plt.hist(hdu_rosat.data.flatten(),bins=500)
plt.yscale('log')
plt.xscale('log')

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(7,7))
ax = fig.add_axes([0.15, 0.1, 0.8, 0.8], projection=wcs_rosat)
ax.imshow(hdu_rosat.data, vmin=0, vmax=6e2, cmap=plt.cm.gray_r,
          origin='lower');
overlay = ax.get_coords_overlay('fk5')
ax.coords['glon'].set_ticks(color='white')
ax.coords['glat'].set_ticks(color='white')

ax.coords['glon'].set_axislabel('Galactic Longitude')
ax.coords['glat'].set_axislabel('Galactic Latitude')

ax.coords.grid(color='yellow', linestyle='solid', alpha=1, lw=2)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(7,7))
ax = fig.add_axes([0.15, 0.1, 0.8, 0.8], projection=wcs_rosat)
ax.imshow(hdu_rosat.data, vmin=0, vmax=6e2, cmap=plt.cm.gray_r,
          origin='lower');
overlay = ax.get_coords_overlay('fk5')

overlay['ra'].set_ticks(color='white')
overlay['dec'].set_ticks(color='white')

overlay['ra'].set_axislabel('Right Ascension')
overlay['dec'].set_axislabel('Declination')

overlay.grid(color='yellow', linestyle='solid', alpha=1, lw=2)

### Exercise

 - Get the 2 MASS catalog from previous session and plot the galaxies over the ROSAT data.
 - Change colors and labels to something nicer.
 - Look at the other data sets that we have loaded at the top create plots for them too.